hiv_text_data pulled from the ref api, using REF_case_study_data script. <br/>
Contains ImpactDetails from all case studies for which "HIV" is a keyword. <br/>
<br/>
This script is from Siddharth Yadav's outstanding kernal:<br/>
https://www.kaggle.com/thebrownviking20/topic-modelling-with-spacy-and-scikit-learn/notebook

In [ ]:
import pandas as pd
from tqdm import tqdm
import string
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [18]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

nlp = spacy.load('en_core_web_sm')

In [28]:
data = pd.read_csv('hiv_text_data.csv')

In [20]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [21]:
tqdm.pandas()
data["processed"] = data["ImpactDetails"].progress_apply(spacy_tokenizer)

100%|██████████| 146/146 [00:09<00:00, 16.54it/s]


In [22]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data["processed"])

In [23]:
NUM_TOPICS = 10

In [24]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=5, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [25]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [26]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('hiv', 279.6359168808762), ('test', 88.59832003290613), ('health', 72.9590564416475), ('people', 71.94492453960335), ('policy', 71.37378875244761), ('report', 62.09214677631603), ('malaria', 58.19092906672823), ('london', 50.62048403487438), ('work', 50.366889225698046), ('change', 49.818447771521164)]
Topic 1:
[('hiv', 203.55876789777057), ('use', 145.88142298804195), ('guideline', 127.31140323188126), ('treatment', 120.48253099035894), ('health', 120.37880544430664), ('policy', 106.5598256775592), ('patient', 103.79411258820907), ('lshtm', 86.67028175845391), ('work', 79.65143659857941), ('aids', 77.47869122398171)]
Topic 2:
[('nepal', 7.108082978603607), ('msm', 6.457391152596684), ('men', 4.692062620660786), ('disease', 4.373324897310977), ('work', 4.240241781775025), ('programme', 4.23976766819603), ('type', 3.8869415615390843), ('qualitative', 3.843875266100945), ('patient', 3.8392166048666136), ('health', 3.3801735854028703)]
Topic 3:
[('test', 1.9057939250

In [27]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      -71.752899  -13.276399       1        1  38.252759
1       69.937531  -65.661331       2        1  27.961928
8       10.770294  -22.875404       3        1  15.105265
0      -15.614913   41.379498       4        1   8.486885
4       92.133728   91.979095       5        1   4.607581
6      -76.369385   76.963814       6        1   4.341054
2       53.269897   31.977194       7        1   0.658122
5      -22.831188  -85.135437       8        1   0.395235
7       10.593602  111.486977       9        1   0.165105
3      118.295250    0.881725      10        1   0.026066, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
705   Default  632.000000            hiv  632.000000  30.0000  30.0000
1583  Default  594.000000            use  594.000000  29.0000  29.0000
471   Default  253.000000           drug  253.000000  28.0000  28.0000
692   Default  735.000000         health  735.000000  27.0000  27.0000
1509  Default  363.000000           test  363.000000  26.0000  26.0000
1079  Default  245.000000        patient  245.000000  25.0000  25.0000
255   Default  251.000000       clinical  251.000000  24.0000  24.0000
1120  Default  493.000000         policy  493.000000  23.0000  23.0000
1188  Default  339.000000      programme  339.000000  22.0000  22.0000
1549  Default  252.000000      treatment  252.000000  21.0000  21.0000
1645  Default  449.000000           work  449.000000  20.0000  20.0000
1374  Default  345.000000        service  345.000000  19.0000  19.0000
732   Default  456.000000         impact  456.000000  18.0000  18.0000
261   Default  107.000000            cmv  107.000000  17.0000  17.0000
56    Default  146.000000         africa  146.000000  16.0000  16.0000
67    Default  192.000000           aids  192.000000  15.0000  15.0000
420   Default  322.000000    development  322.000000  14.0000  14.0000
744   Default  363.000000        include  363.000000  13.0000  13.0000
974   Default  267.000000       national  267.000000  12.0000  12.0000
682   Default  219.000000      guideline  219.000000  11.0000  11.0000
207   Default  186.000000           care  186.000000  10.0000  10.0000
1505  Default  170.000000     technology  170.000000   9.0000   9.0000
988   Default  301.000000            new  301.000000   8.0000   8.0000
1282  Default  271.000000         report  271.000000   7.0000   7.0000
1084  Default  232.000000         people  232.000000   6.0000   6.0000
1207  Default  207.000000        provide  207.000000   5.0000   5.0000
1408  Default  131.000000          south  131.000000   4.0000   4.0000
1577  Default  181.000000     university  181.000000   3.0000   3.0000
359   Default  229.000000        country  229.000000   2.0000   2.0000
445   Default  136.000000        disease  136.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
948   Topic10    0.030397        million  137.642067  -0.1658  -6.1716
445   Topic10    0.030182        disease  136.948428  -0.1678  -6.1787
1207  Topic10    0.035102        provide  207.389849  -0.4318  -6.0277
1293  Topic10    0.017781     resistance   39.090075   0.5568  -6.7078
1268  Topic10    0.020265         relate   60.294414   0.2542  -6.5771
255   Topic10    0.033634       clinical  251.066795  -0.6657  -6.0704
641   Topic10    0.010933           gain   11.198899   1.3205  -7.1942
1505  Topic10    0.027862     technology  170.635307  -0.4677  -6.2587
1374  Topic10    0.035086        service  345.606164  -0.9430  -6.0281
152   Topic10    0.028329           base  195.361745  -0.5864  -6.2420
201   Topic10    0.016373         cancer   37.340562   0.5201  -6.7903
1655  Topic10    0.026218           year  158.080822  -0.4521  -6.3195
847   Topic10    0.017729          large   50.136709   0.3050  -6.